Here's wherer we mint. We will explore all the prompt methodologies, in time...

In [49]:
import pandas as pd

# Load the cleaned DataFrame from parquet file
cleaned_df = pd.read_parquet('02-cleaner-data/cleaned_data.parquet')

# lets drop bloomberg for now
cleaned_df = cleaned_df[cleaned_df['source'] != 'bloomberg.com']

cleaned_df.head()

,url,text,isMorning,source,rank
date,,,,,
2021-02-07,https://www.ft.com/content/e9bbd4fe-e6bf-4383-...,Oxford/AstraZeneca jab fails to prevent mild a...,False,ft.com,1
2021-02-07,https://www.ft.com/content/f2e8f0d6-db17-42af-...,Antony Blinken blasts China in first phone call,False,ft.com,2
2021-02-07,https://www.ft.com/content/6d86951c-a4af-4d0b-...,The UK companies seeing the upside of Brexit,False,ft.com,3
2021-02-07,https://www.ft.com/content/97e43c01-2b5f-4907-...,Government to take more control in overhaul of...,False,ft.com,4
2021-02-07,https://www.ft.com/content/a0731e22-6407-4a52-...,Remote workers tap scheme to see out pandemic ...,False,ft.com,5


In [50]:
def generate_daily_headlines_for_llm(day):
    # Filter the DataFrame for the given day
    day_headlines = cleaned_df.loc[day]
    
    # Create a string to hold the headlines and the date
    headlines_str = f"Date: {day}\n"
    
    # Get unique sources
    unique_sources = day_headlines['source'].unique()
    
    # Iterate through the sources and add top 10 headlines from each source to the string
    for source in unique_sources:
        headlines_str += f"Source: {source}\n"
        top_10_headlines = day_headlines[day_headlines['source'] == source].head(15)
        for index, row in top_10_headlines.iterrows():
            headlines_str += f"- {row['text']}\n"
    
    return headlines_str

example_day = '2020-01-23'
print(generate_daily_headlines_for_llm(example_day))


Date: 2020-01-23
Source: ft.com
- China extends travel restrictions in effort to contain coronavirus
- China quarantines city in effort to contain viral outbreak
- Trade tensions erupt between London and Washington
- How China’s slow response aided coronavirus outbreak
- Boris Johnson hails moment as EU withdrawal bill clears parliament
- Tesla hits $100bn stock market value for first time
- Iran: the unspoken battle to succeed Ayatollah Khamenei
- Trade tensions erupt between London and Washington
- US and France agree deal on digital tax
- US Treasury secretary Mnuchin launches attack on Greta Thunberg
- Tesla hits $100bn stock market value for first time
- ‘Mr Super Prime’ resigns after posting luxury home on Instagram
- Boeing chief on the Max: ‘I believe in this airplane’
- Comcast takes aim at CNN with NBC-Sky global news channel
- Amazon Music subscriber numbers close in on Apple
Source: drudgereport.com
- Dems face risks and limits in impeachment trial...
- Senators Play With T

Cool, now let's pair it with the target and make a prompt!

In [51]:
import pandas as pd

def load_targets_csv():
    targets_df = pd.read_csv('04-targets/targets.csv', parse_dates=['Date'], index_col='Date')
    return targets_df

targets_df = load_targets_csv()
targets_df.head()

def get_ticker_for_date(date):
    try:
        ticker = targets_df.loc[date, 'Ticker']
    except KeyError:
        ticker = None
    return ticker


In [52]:

# Generate a DataFrame with date as index, and columns for prompt and target

# Create a list of dates for which we have data
dates = cleaned_df.index.unique()

# Initialize an empty list to hold the prompt-target pairs
prompt_target_pairs = []

# Loop through each date to generate prompts and targets
for date in dates:
    # Generate the daily headlines as the prompt
    prompt = generate_daily_headlines_for_llm(date)
    
    target = get_ticker_for_date(date)
    # Append the date, prompt, and target as a tuple to the list
    prompt_target_pairs.append((date, prompt, target))

# Convert the list of tuples into a DataFrame
prompt_target_df = pd.DataFrame(prompt_target_pairs, columns=['date', 'prompt', 'target'])

# Set the date as the index
prompt_target_df.set_index('date', inplace=True)

prompt_target_df

# Count how many rows have target None
num_rows_with_target_none = prompt_target_df[prompt_target_df['target'].isnull()].shape[0]
print(f"Number of rows with target None: {num_rows_with_target_none}")

# Count how many rows have prompt None
num_rows_with_prompt_none = prompt_target_df[prompt_target_df['prompt'].isnull()].shape[0]
print(f"Number of rows with prompt None: {num_rows_with_prompt_none}")

# drop rows with prompt or target None
prompt_target_df = prompt_target_df.dropna(subset=['prompt', 'target'])

# Split the data into train and test
train_df = prompt_target_df[prompt_target_df.index < '2024-01-01']
test_df = prompt_target_df[prompt_target_df.index >= '2024-01-01']

# save them as jsonl and csv
if not os.path.exists('05-prompts'):
    os.makedirs('05-prompts')
train_df.to_csv('05-prompts/train.csv')
test_df.to_csv('05-prompts/test.csv')
train_df.to_json('05-prompts/train.json', orient='records', lines=True)
test_df.to_json('05-prompts/test.json', orient='records', lines=True)
# rename to jsonl
os.rename('05-prompts/train.json', '05-prompts/train.jsonl')
os.rename('05-prompts/test.json', '05-prompts/test.jsonl')


Number of rows with target None: 155
Number of rows with prompt None: 0


,prompt,target
date,,


# Free. Gold.